In [1]:
%matplotlib qt4
from __future__ import division

from models import tools, optimize, models, filters
from models.tests import PerformanceTest

import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
data = tools.load_data(offset=5e6, limit=1e6)
data = data[filters.sequentize(data)]
print len(data)

Loaded 961218 answers.
144188


## Elo Model

In [5]:
elo_test = PerformanceTest(models.EloModel(), data, split_data=True)
elo_test.run()
elo_test.results['train']

RMSE: 0.423472181742
AUC: 0.708818093631
LL: -12401.6142926
OFF: 0.00353807736108
CORRECT: 16978
ACCURACY: 0.73536036036
Set Size: 23088

## Elo With Response Time

In [6]:
elot_test = PerformanceTest(models.EloResponseTime(), data, split_data=True)
elot_test.run()
elot_test.results['train']

RMSE: 0.428191222313
AUC: 0.703571244911
LL: -12709.1073088
OFF: -0.0139681485231
CORRECT: 16865
ACCURACY: 0.730466042966
Set Size: 23088

## PFA

In [9]:
pfa = models.PFAModel(models.EloModel(), gamma=1.022, delta=-0.0806)
pfa_test = PerformanceTest(pfa, data)
pfa_test.run()
pfa_test.results['train']

RMSE: 0.365139978121
AUC: 0.753209871305
LL: -inf
OFF: 0.00202036501965
CORRECT: 118504
ACCURACY: 0.821871445613
Set Size: 144188

## PFA/E

In [4]:
pfae = models.PFAExt(models.EloModel(), gamma=2.614, delta=-0.642)
pfae_test = PerformanceTest(pfae, data)
pfae_test.run()
pfae_test.results['train']

RMSE: 0.345625192694
AUC: 0.799925501439
LL: -55503.2481574
OFF: -0.0164008439892
CORRECT: 120510
ACCURACY: 0.835783837767
Set Size: 144188

## PFA/G

In [9]:
pfag = models.PFAGong(models.EloModel(), gamma=1.798, delta=0.091, decay=0.425)
pfag_test = PerformanceTest(pfag, data)
pfag_test.run()
pfag_test.results['train']

RMSE: 0.349664345959
AUC: 0.782478530629
LL: -58420.4505052
OFF: 0.0218006882277
CORRECT: 120477
ACCURACY: 0.8355549699
Set Size: 144188

## PFA/E/T

### Poly

In [6]:
pfaet = models.PFAExtTiming(models.EloModel(), gamma=2.004, delta=-0.713,
                            time_effect_fun='poly', a=2.931, c=0.27)
pfaet_test = PerformanceTest(pfaet, data)
pfaet_test.run()
pfaet_test.results['train']

RMSE: 0.340610555244
AUC: 0.806426282763
LL: -54082.4963446
OFF: 0.00152109514334
CORRECT: 121341
ACCURACY: 0.841547146781
Set Size: 144188

### Log

In [31]:
pfaet = models.PFAExtTiming(models.EloModel(), gamma=1.906, delta=-0.806,
                            time_effect_fun='log', a=1.789, c=0.128)
pfaet_test = PerformanceTest(pfaet, data)
pfaet_test.run()
pfaet_test.results['train']

RMSE: 0.340689226279
AUC: 0.806618196313
LL: -inf
OFF: -0.00167219711391
CORRECT: 121301
ACCURACY: 0.841269731184
Set Size: 144188

### Exp

In [26]:
pfaet = models.PFAExtTiming(models.EloModel(), gamma=2.006, delta=-0.757,
                            time_effect_fun='exp', a=1.005, c=0.009)
pfaet_test = PerformanceTest(pfaet, data)
pfaet_test.run()
pfaet_test.results['train']

RMSE: 0.341023057373
AUC: 0.808491437126
LL: -54145.6164025
OFF: -0.00988678960385
CORRECT: 121099
ACCURACY: 0.839868782423
Set Size: 144188

### Staircase

In [8]:
staircase = {
    (0, 60): 1.43,
    (60, 90): 1.17,
    (90, 150): 1.01,
    (150, 300): 0.93,
    (300, 600): 0.82,
    (600, 60*30): 0.78,
    (60*30, 60*60*3): 0.76,
    (60*60*3, 60*60*24): 0.63,
    (60*60*24, 60*60*24*5): 0.42,
    (60*60*24*5, np.inf): 0.12,
}

pfaest = models.PFAExtStaircase(models.EloModel(), staircase=staircase,
                                gamma=1.814, delta=-0.827)
pfaest_test = PerformanceTest(pfaest, data)
pfaest_test.run()
pfaest_test.results['train']

RMSE: 0.340814661115
AUC: 0.807544282931
LL: -54114.2313641
OFF: -0.00774427081198
CORRECT: 121218
ACCURACY: 0.840694093822
Set Size: 144188

In [7]:
staircase2 = tools.connect_points(
    [(0, 1.6), (45, 1.43), (75, 1.17), (120, 1.01), (225, 0.93), (1200, 0.78),
     (6300, 0.76), (48600, 0.63), (259200, 0.42), (432000, 0.12), (10e10, 0.02)]
)
pfaest2 = models.PFAExtTiming(models.EloModel(), time_effect_fun=staircase2,
                              gamma=1.814, delta=-0.827)
pfaest2_test = PerformanceTest(pfaest2, data)
pfaest2_test.run()
pfaest2_test.results['train']

RMSE: 0.340730575822
AUC: 0.807535789235
LL: -54090.3633182
OFF: -0.00669204286735
CORRECT: 121238
ACCURACY: 0.84083280162
Set Size: 144188

## PFA/G/T

### Poly

In [12]:
pfagt = models.PFAGongTiming(models.EloModel(), gamma=1.973, delta=0.244,
                             time_effect_fun='poly', a=2.007, c=0.231)
pfagt_test = PerformanceTest(pfagt, data)
pfagt_test.run()
pfagt_test.results['train']

RMSE: 0.350450285316
AUC: 0.778158513974
LL: -60650.6500305
OFF: 0.0104245574738
CORRECT: 120218
ACCURACY: 0.833758703914
Set Size: 144188

### Log

In [18]:
pfagt = models.PFAGongTiming(models.EloModel(), gamma=0.837, delta=0.048,
                             time_effect_fun='log', a=1.918, c=0.128)
pfagt_test = PerformanceTest(pfagt, data)
pfagt_test.run()
pfagt_test.results['train']

RMSE: 0.350875856909
AUC: 0.780122469727
LL: -60531.4924388
OFF: 0.00697782295559
CORRECT: 119482
ACCURACY: 0.832545953705
Set Size: 143514

### Exp

In [21]:
pfagt = models.PFAGongTiming(models.EloModel(), gamma=1.297, delta=0.198,
                             time_effect_fun='log', a=0.824, c=0.0023)
pfagt_test = PerformanceTest(pfagt, data)
pfagt_test.run()
pfagt_test.results['train']

RMSE: 0.357845249766
AUC: 0.746413216867
LL: -78109.603908
OFF: 0.0466174867108
CORRECT: 119913
ACCURACY: 0.835549145031
Set Size: 143514

### PFA/E/RT

In [3]:
pfart = models.PFAResponseTime(models.EloModel(), gamma=1.453, delta=-1.356, zeta=1.886)
pfart_test = PerformanceTest(pfart, data)
pfart_test.run()
pfart_test.results['train']

RMSE: 0.345609041461
AUC: 0.799554645876
LL: -57336.4888702
OFF: -0.00122789889836
CORRECT: 120262
ACCURACY: 0.83406386107
Set Size: 144188